In [1]:
import java.io.File
import com.lignting.neural.*
import org.jetbrains.kotlinx.multik.api.mk
import org.jetbrains.kotlinx.multik.api.ndarray

In [2]:
val trainPath = "res/used_car_train_20200313.csv"
val testPath = "res/used_car_testA_20200313.csv"

In [3]:
fun readData(path: String): List<Pair<List<Double?>, Double?>> {
    val data = File(path).readLines().drop(1)
        .map { it.split(" ").map { it.trim() } }
        .map { it.map { it.toDoubleOrNull() } }
        .map { it.toMutableList() }
        .map { it to it.removeAt(15) }
    return data
}

In [4]:
val trainData = readData(trainPath)
val testData = readData(testPath)

In [5]:
val xNullCounter = MutableList(trainData[0].first.size) { 0 }
var yNullCounter = 0
trainData.forEach {
    it.first.mapIndexed { index, value -> index to if (value == null) 1 else 0 }.forEach {
        xNullCounter[it.first] += it.second
    }
    if (it.second == null)
        yNullCounter++
}
xNullCounter to yNullCounter

([0, 0, 0, 1, 0, 4506, 8680, 5981, 0, 0, 24324, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0)

In [6]:
fun fillna(data: List<Pair<List<Double?>, Double?>>) =
    data.map {
        it.first.map {
            it ?: -1.0
        } to (it.second ?: -1.0)
    }

In [7]:
val proceedTrainData = fillna(trainData)
val processedTestData = fillna(testData)

In [8]:
val trainX = mk.ndarray(proceedTrainData.map { it.first })
val trainY = mk.ndarray(proceedTrainData.map { listOf(it.second) })
val testX = mk.ndarray(processedTestData.map { it.first })
val testY = mk.ndarray(processedTestData.map { listOf(it.second) })

In [12]:
val model = Model(
    Dense(30, 20),
    Dense(20, 12),
    Dense(12, 8),
    Dense(8, 4),
    Dense(4, 1),
    loss = Mae(),
    learningRate = 0.0001
)

In [13]:
(0..3).forEach {
    val loss = model.fit(trainX, trainY)
    println("train times = $it, loss = $loss")
}

train times = 0, loss = 1.1706309612141897E10
train times = 1, loss = 3.998968777408792E16
train times = 2, loss = 1.12821890775108211E18
train times = 3, loss = 8.209179629284521E18
